<a href="https://colab.research.google.com/github/buganart/unagan/blob/master/unagan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
# @markdown 1. Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`
# @markdown 2. Check GPU, should be a Tesla V100 if you want to train it as fast as possible.
# @markdown 3. Mount google drive.
# @markdown 4. Log in to wandb.


!nvidia-smi -L
import os

print(f"We have {os.cpu_count()} CPU cores.")
print()

try:
    from google.colab import drive, output

    IN_COLAB = True
except ImportError:
    from IPython.display import clear_output

    IN_COLAB = False

from pathlib import Path

if IN_COLAB:
    drive.mount("/content/drive/")

    if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
        raise RuntimeError(
            "Shortcut to our shared drive folder doesn't exits.\n\n"
            "\t1. Go to the google drive web UI\n"
            '\t2. Right click shared folder IRCMS_GAN_collaborative_database and click "Add shortcut to Drive"'
        )

clear = output.clear if IN_COLAB else clear_output


def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        clear()
        print(msg)


print()
print("Wandb installation and login ...")
%pip install -q wandb

wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login

In [ ]:
#@title Configuration

# Fill in the configuration then Then, select `Runtime` and `Run all` then let it ride!

#@markdown #### Training

drive = Path('/content/drive/MyDrive')
drive_audio_db_root = drive / "AUDIO DATABASE"
collaborative_database = drive / "IRCMS_GAN_collaborative_database"
violingan_experiment_dir = collaborative_database / "Experiments" / "colab-violingan"
experiment_dir = violingan_experiment_dir / "unagan"

#@markdown The path of the Audio Database you'd like to work with
#@markdown (relative to `My Drive/PUBLICATIONS/The Replicant/AUDIO DATABASE/`)
audio_db = "RAW Sessions/Roberto Studio Material 22050" #@param {type:"string"}
audio_db_dir = drive_audio_db_root / audio_db
if not audio_db_dir.exists():
    print(f"Error: audio_db_dir {audio_db_dir} does not exist.")

#@markdown [Optional] ID of wandb run to resume.
resume_run_id = "" #@param {type: "string"}

def check_wandb_id(run_id):
    if run_id and not re.match(r"^[\da-z]{8}$", run_id):
        raise RuntimeError(
            "Run ID needs to be 8 characters long and contain only letters a-z and digits.\n"
            f"Got \"{run_id}\""
        )

check_wandb_id(resume_run_id)

#@markdown #### Generation (Todo: should we move this to separate notebook?)
#@markdown The location of your Melgan Output (from within "My Drive/")
melgan_output_location = "RobertoStudioMaterial10-29/" #@param {type:"string"}
#@markdown The output name you'd like to give the Unagan directory (e.g. "Scelsi1")
unagan_output_location = "RobertoStudioMaterial10-29-Unagan-Output/" #@param {type:"string"}

config = dict(
    audio_db_dir=audio_db_dir,
    melgan_output_location=melgan_output_location,
    unagan_output_location=unagan_output_location,
    resume_run_id=resume_run_id,
)
for k,v in config.items():
    print(f"=> {k:30}: {v}")

In [ ]:
#@title Clone unagan repo.
if IN_COLAB:
    !git clone https://github.com/buganart/unagan
    clear_on_success("Repo cloned!")

In [ ]:
#@title Install dependencies
if IN_COLAB:
    %cd "/content/unagan/"
    
%pip install -q -U -r requirements.txt
clear_on_success("Dependencies installed!")

In [ ]:
#@title Copy files to local runtime.
local_wav_dir = Path("data")
local_wav_dir.mkdir(exist_ok=True)
!find "{audio_db_dir}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$local_wav_dir"/
clear_on_success("All files copied to this runtime.")

audio_paths = sorted(list(local_wav_dir.glob("*")))
num_files = len(audio_paths)
print(f"Database has {num_files} files in total.")

In [ ]:
#@title Collect audio clips.
!python scripts/collect_audio_clips.py --audio-dir "$local_wav_dir" --extension wav
clear_on_success(f"Done.")

In [ ]:
#@title Extract mel spectrograms.
!python scripts/extract_mel.py 
clear_on_success("Done!")

In [ ]:
#@title Make dataset
!python scripts/make_dataset.py
clear_on_success("Done!")

In [ ]:
#@title Compute mean and standard deviation
!python scripts/compute_mean_std.mel.py
clear_on_success("Done!")

In [ ]:
#@title Train
!env PYTHONPATH="." python scripts/train.hierarchical_with_cycle.py \
    --model-id "$resume_run_id" \
    --wandb-dir "$experiment_dir"

Epoch 1. Batch: 110/500, T: 0.479, (tr) G: 0.51602, (tr) D: 0.83547, (tr) RealD: 0.83570, (tr) FakeD: 0.51602, (tr) Convergence: 1.15538, (tr) NoiseRecon: 0.57139, (tr) RealRecon: 1.02191,  0.0036522340960800648
Epoch 1. Batch: 120/500, T: 0.463, (tr) G: 0.63555, (tr) D: 0.59137, (tr) RealD: 0.59137, (tr) FakeD: 0.63555, (tr) Convergence: 0.63555, (tr) NoiseRecon: 0.57006, (tr) RealRecon: 0.62287,  0.0
Epoch 1. Batch: 130/500, T: 0.470, (tr) G: 0.53286, (tr) D: 0.44853, (tr) RealD: 0.44853, (tr) FakeD: 0.53286, (tr) Convergence: 0.53286, (tr) NoiseRecon: 0.54046, (tr) RealRecon: 0.58507,  0.0
Epoch 1. Batch: 140/500, T: 0.457, (tr) G: 0.48762, (tr) D: 0.36703, (tr) RealD: 0.36703, (tr) FakeD: 0.48762, (tr) Convergence: 0.48762, (tr) NoiseRecon: 0.51662, (tr) RealRecon: 0.51641,  0.0
Epoch 1. Batch: 150/500, T: 0.450, (tr) G: 0.47839, (tr) D: 0.43835, (tr) RealD: 0.44087, (tr) FakeD: 0.47839, (tr) Convergence: 0.47839, (tr) NoiseRecon: 0.53826, (tr) RealRecon: 0.59810,  0.00488998740911

# Anything below is untested and probably won't work anymore.

In [ ]:
%mkdir -p "/content/unagan/models/custom/vocoder"

In [ ]:
unagan_output_dir = "/content/drive/My Drive/AUDIO DATABASE/UNAGAN OUTPUT/" + unagan_output_location
%mkdir -p "$unagan_output_dir"

In [ ]:
cp "/content/unagan/training_data/exp_data/mean.mel.npy" "/content/unagan/models/custom"

In [ ]:
cp "/content/unagan/training_data/exp_data/mean.mel.npy" "$unagan_output_dir"

In [ ]:
cp "/content/unagan/training_data/exp_data/std.mel.npy" "/content/unagan/models/custom"

In [ ]:
cp "/content/unagan/training_data/exp_data/std.mel.npy" "$unagan_output_dir"

In [ ]:
import fnmatch
import os

for folder in os.listdir("/content/unagan/wandb/"):
    if folder.startswith ('run-'):
        print (folder)
        run_dir = folder

In [ ]:
unagan_param = "/content/unagan/wandb/" + run_dir + "/files/model/params.Generator.latest.torch"

In [ ]:
cp "$unagan_param" "/content/unagan/models/custom/params.generator.hierarchical_with_cycle.pt"

In [ ]:
cp "$unagan_param" "$unagan_output_dir"

In [ ]:
melgan_args = "/content/drive/My Drive/" + melgan_output_location + "args.yml"
melgan_modules = "/content/drive/My Drive/" + melgan_output_location + "modules.py"
melgan_netG = "/content/drive/My Drive/" + melgan_output_location + "best_netG.pt"

In [ ]:
cp "$melgan_args" "/content/unagan/models/custom/vocoder/"

In [ ]:
cp "$melgan_modules" "/content/unagan/models/custom/vocoder/"

In [ ]:
cp "$melgan_netG" "/content/unagan/models/custom/vocoder/params.pt"

In [ ]:
!env PYTHONPATH="$PWD" python generate.py --gid 0 --data_type custom --arch_type hc --duration 600 --num_samples 60

In [ ]:
!rsync -avP "/content/unagan/generated_samples/custom.hierarchical_with_cycle" "$unagan_output_dir"